In [2]:
import smtplib
from email.message import EmailMessage
import imghdr
from datetime import datetime
from config import imap_password_customplanet, db_username, db_password
import pandas as pd
import numpy as np
from modules.db_operations import DatabaseConnector


pd.set_option('display.max_columns', None)
#This can be further cleansed in the future
df = pd.read_csv('KC SCHOOLS - KC SCHOOLS.csv')

EMAIL_ADDRESS = 'team@customplanet.com'
EMAIL_PASS = imap_password_customplanet
contacts = ['2015samtaylor@gmail.com']

# -----------------------------------------------------------------------function to send email----------------------------

def send_mail(school, sport, png_path, email):
    
    msg = EmailMessage()
    msg['Subject'] = f'Local {sport} Jerseys for {school}'
    msg['From'] = EMAIL_ADDRESS
    # msg['To'] = (contact for contact in contacts)
    msg['To'] = email

    #Create the body of the email by setting the content

    body = f'''
            <!DOCTYPE html>
        <html>
        <head>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    background-color: #f4f4f4;
                    margin: 0;
                    padding: 0;
                }}

                .container {{
                    background-color: #ffffff;
                    max-width: 600px;
                    margin: 0 auto;
                    padding: 20px;
                }}

                h1 {{
                    color: #007BFF;
                }}

                p {{
                    font-size: 16px;
                    line-height: 1.6;
                }}

                a {{
                    color: #007BFF;
                    text-decoration: none;
                }}

                a:hover {{
                    text-decoration: underline;
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Hello {school}!</h1>
                <p>We are a local supplier for Custom {sport} Jerseys. How would you like to check out what we have?</p>
                <p>We can put your logo on just about anything!</p>
                <p>Check out our website or give us a call to get a free sample at <a href="http://customplanet.com">customplanet.com</a>.</p>
            </div>
        </body>
        </html>
    '''
    
    #subtype must be indicated if it is HTML
    msg.set_content(body, subtype='html')
    
    #insert School logos into the Email thread from the PNG folder
    try:
        with open(f'PNGs/{png_path}.png', 'rb') as f:
            file_data = f.read()
            file_type = imghdr.what(f.name)
            file_name = f.name
            msg.add_attachment(file_data, maintype = 'image', subtype = file_type, filename = file_name)

    except FileNotFoundError:
        pass


    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(EMAIL_ADDRESS, EMAIL_PASS)
        smtp.send_message(msg)
        

#Temporary testing to modify the Athletic Director email to my own
df.loc[0, 'Athletic Director.1'] = '2015samtaylor@gmail.com'
df.loc[1, 'Athletic Director.1'] = 'sammytaylor2006@gmail.com'
df.loc[2, 'Athletic Director.1'] = '2015samtaylor@gmail.com'
df.loc[3, 'Athletic Director.1'] = 'sammytaylor2006@gmail.com'


def process(contact_column, sport, email_campaign_name):

    data_list = []
    processed_emails = set()

    for index, row in df[:3].iterrows():

        # Get the current date
        now = datetime.now()
        formatted_date = now.strftime("%m%d%Y%H%M%S")

        school = row['School Name']
        png_path = row['PNG_PATH']
        contact = row[contact_column]

        # Check if the email has already been processed
        if contact in processed_emails:
            print(f"Skipping email to {contact} as it has already been processed.")
            continue

        data = [school, contact, sport, formatted_date]
        data_list.append(data)

        # Mark the email as processed
        processed_emails.add(contact)

        send_mail(school, sport, png_path, contact)

    
    data_list = pd.DataFrame(data_list, columns=['School', 'Contact', 'Sport', 'Date_Sent'])

    data_list.rename(columns = {'Contact': 'contact_email', 'School': 'school', 'Date_Sent': 'date_sent', 'Sport': 'sport'}, inplace = True)
    data_list['date_sent'] = data_list['date_sent'].astype(str)

    data_list['contact_position'] = contact_column
    data_list['sent_from'] = EMAIL_ADDRESS
    data_list['email_campaign'] = email_campaign_name
    data_list['recipient_id'] = data_list['contact_email'] + '_' + data_list['date_sent'] #composite key
    data_list['date_sent'] = now.strftime("%Y-%m-%d %H:%M:%S")

    data_list.to_csv('output.csv', index=False)

    return(data_list)

# #Must be the column name of the position wanting to send to
email_history = process('Athletic Director.1', 'Basketball', 'Initial')

# Specify database connection details
server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
database = 'emailcampaign'
db = 'emailcampaign'
table_name = 'email_history'

# Instantiate the DatabaseConnector class, and send over the dataframe via append
db_connector = DatabaseConnector(server, database, db_username, db_password, db, table_name)
db_connector.send(email_history)

Skipping email to 2015samtaylor@gmail.com as it has already been processed.


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Sending_Emails\modules\db_operations.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)


In [3]:
email_history

,school,contact_email,sport,date_sent,contact_position,sent_from,email_campaign,recipient_id
0,F.L. Schlagle High School (KCKS),2015samtaylor@gmail.com,Basketball,2023-11-26 22:35:34,Athletic Director.1,team@customplanet.com,Initial,2015samtaylor@gmail.com_11262023223528
1,Plaza Middle (Park Hill),sammytaylor2006@gmail.com,Basketball,2023-11-26 22:35:34,Athletic Director.1,team@customplanet.com,Initial,sammytaylor2006@gmail.com_11262023223532


In [ ]:
#Why does the date keep defaulting to 11/26 ? The only date that is not on there? 
# I think it is because it defaults to the original thread date?
# The date sent does take what is recent because it is hedged on the original thread based on that contact_email and subject line. 
#Think through this one

# To DO

In [ ]:
#Put the PNG on the given sport jersey
#Clean the CSV to iterate properly, this should be done. 


#Think about how the scrape is going to work in thread. 
#There is data within there right now. What if the foreign key, does not match?
#Need some SSIS scripts to drop duplicates, etc... more focus on SSIS once data is in

In [ ]:
import pandas as pd

df = pd.read_csv(r'C:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Sending_Emails\KC SCHOOLS - KC SCHOOLS.csv')

In [ ]:
df